In [1]:
from classes import *
from helper_functions import *

import pandas as pd
import numpy as np
import datetime, time
import random
random.seed(2022)

# for configurations
from dotenv import load_dotenv
import sys, os, re, math
load_dotenv()

# for distance matrix
import gmaps
import googlemaps
from haversine import haversine
from scipy.spatial.distance import cdist

# for routing
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import warnings
warnings.filterwarnings('ignore')

In [2]:
## DATA HANDLING
file_path = 'data/processed_data_32.csv'
num_vehicles = 5
ins_ends_coords = [(-34.8218243,138.7292797),(-34.8104796,138.6111791),(-34.8938435,138.6918266),(-34.7825552,138.610732),(-34.8516117,138.6722955)]

In [3]:
## CONFIGURATION 
# Importing the dataset and configuration parameters
data = pd.read_csv(file_path)
API_KEY = os.getenv("API_KEY")
FACTORY_GEO_COORD = os.getenv("FACTORY_GEO_COORD") 
factory_coord = list(map(float,FACTORY_GEO_COORD.split(',')))
# enter your api key here
gmaps.configure(api_key=API_KEY)

In [5]:
## FILTERING ONLY 1 INSTALLER JOBS
jobs_installer_1 = data[data["installers_required"]==1].reset_index(drop=True)

In [6]:
## PROCESSING DATA FOR VRP USE
df_pending = jobs_installer_1[['id','Latitude','Longitude']]
# Inserting factory location to top
new_row = pd.DataFrame({'id':0, 'Latitude':factory_coord[0],'Longitude':factory_coord[1]}, index =[0])
df_pending = pd.concat([new_row, df_pending]).reset_index(drop=True)
df_pending.set_index('id', inplace=True)
df_pending['Latitude'] = df_pending['Latitude'].astype(float)
df_pending['Longitude'] = df_pending['Longitude'].astype(float)

# Inserting installers' end locations to df_pending
for i in range(len(ins_ends_coords)):
    ins_id = i+1
    df_pending.loc[ins_id] = [ins_ends_coords[i][0],ins_ends_coords[i][1]]

In [7]:
# SETTING UP variables and figures for VISUALISATION
jobs, ins_ends = [],[]
for i, row in df_pending.iterrows():
    if i == 0:
        continue
    elif i//100 == 0: # Expecting max 100 installers for the program and job_ids no less than 100
        ins_end = { 'id': str(i), 'location': (float(row['Latitude']), float(row['Longitude']))  }
        ins_ends.append(ins_end)
    else:
        job = { 'id': str(i), 'location': (float(row['Latitude']), float(row['Longitude']))  }
        jobs.append(job)

factory = {'location': (factory_coord[0],factory_coord[1])}
factory_layer = gmaps.symbol_layer([factory['location']], hover_text='Factory', info_box_content='Factory', fill_color='white', stroke_color='red', scale=6)

job_locations = [job['location'] for job in jobs]
job_labels = [job['id'] for job in jobs]
jobs_layer = gmaps.symbol_layer(
    job_locations, hover_text=job_labels, fill_color='white', stroke_color='black', scale=3
)

ins_end_locations = [ins_end['location'] for ins_end in ins_ends]
ins_labels = [ins_end['id'] for ins_end in ins_ends]
ins_ends_layer = gmaps.symbol_layer(
    ins_end_locations, hover_text=ins_labels, fill_color='white', stroke_color='red', scale=3
)

fig = gmaps.figure()
fig.add_layer(factory_layer)
fig.add_layer(jobs_layer)
fig.add_layer(ins_ends_layer)
# fig

In [9]:
demands, penalties = [0],[0]
pref_dates, pref_days, pref_installers=[None],[None],[None] # first values for depot 
pref_time_windows=[None]
## DEMANDS 
for i in range(len(jobs)):
    if pd.isnull(jobs_installer_1.loc[i,'expected_job_time']):
        demands.append(int(60))
        continue
    demands.append(int(jobs_installer_1.loc[i,'expected_job_time']))
    
## DUE_DATE_DIFFERENCE    
for i in range(len(jobs)):
    due_date = datetime.datetime.strptime(jobs_installer_1.loc[i,'est_installation_date'], '%d/%m/%Y').date()
    curr_date = datetime.date.today()
    due_date_diff = (due_date-curr_date).days
    penalties.append(due_date_diff)
    
## CUSTOMER PREFERRED DATE
for i in range(len(jobs)):
    if pd.isnull(jobs_installer_1.loc[i,'pref_date']):
        pref_dates.append(None)
        continue
    pref_date_to_append = datetime.datetime.strptime(jobs_installer_1.loc[i,'pref_date'], '%d/%m/%Y').date()
    pref_dates.append(pref_date_to_append)
    
## CUSTOMER PREFERRED DAY
for i in range(len(jobs)):
    if pd.isnull(jobs_installer_1.loc[i,'pref_day']):
        pref_days.append(None)
        continue
    pref_day_to_append = int(jobs_installer_1.loc[i,'pref_day'])
    pref_days.append(pref_day_to_append)

## CUSTOMER PREFERRED INSTALLER
for i in range(len(jobs)):
    if np.isnan(jobs_installer_1.loc[i,'pref_installer']):
        pref_installers.append(None)
        continue
    pref_installers.append(int(jobs_installer_1.loc[i,'pref_installer']))

## CUSTOMER PREFERRED TIME WINDOWS
for i in range(len(jobs)):
    if pd.isnull(jobs_installer_1.loc[i,'pref_time_window']):
        pref_time_windows.append(None)
        continue
    curr_pref_time_window = jobs_installer_1.loc[i,'pref_time_window'].split(',')
    curr_pref_time_window[0] = time_to_minutes(curr_pref_time_window[0])
    curr_pref_time_window[1] = time_to_minutes(curr_pref_time_window[1])
    pref_time_windows.append(curr_pref_time_window)

## END LOCATIONS OF INSTALLERS
end_locations = []
for i in range(num_vehicles):
    end_locations.append(len(jobs)+i+1)
    
dist_matrix,time_matrix = get_distance_time_matrices(df_pending)

In [10]:
routes,total_distance,total_load = solve_vrp_for(time_matrix, num_vehicles, demands, penalties, end_locations, pref_dates, pref_days, pref_installers, pref_time_windows)

1 2200
2 3200
3 3100
4 3100
5 3000
6 999999
7 0
8 2400
9 2000
10 3200
11 3200
12 3000
13 3100
14 3100
15 3000
16 3100
17 999999
18 3200
19 1700
20 1000
21 3100
22 0
23 3100
24 3100
Hooray 25
25 999999
26 3100
27 3000
28 3000
29 2600
Hooray 30
30 999999
31 2600
32 2600
33 3200
34 2200
35 3200
36 3200
37 1800
38 3200
39 2000
40 2600
41 2600
42 2600
43 3200
44 3200
45 2200
46 0
47 3200
48 1700
49 0
50 1200
51 3200
52 3200
53 3000
54 3100
55 3000
56 0
57 3100
58 2600
59 3100
60 2600
Hooray 61
61 999999
62 2600
63 2600
64 2500
65 2500
66 3200
67 2500
68 2400
Hooray 69
69 999999
70 0
71 3200
72 2200
73 3200
74 3200
75 2200
76 3200
77 2000
78 0
79 3200
80 1700
81 1700
82 1400
83 3200
84 3200
85 3200
86 1400
87 1300
88 1300
89 3200
90 3200
91 1100
92 0
93 0
94 1000
95 2600
96 2500
97 2200
98 0
99 1700
100 1400
101 1300
102 1200
103 1700
104 3100
105 0
17 120 240
18 0 240
52 240 480
55 240 480
64 240 480
71 0 240
79 240 480
86 240 480
87 0 240
89 240 480
97 240 480
Objective: 164037
Dropped nod

In [12]:
c=8/0 # Just a hack to break the notebook's run here :p
if routes:
    map_solution(factory, jobs, ins_ends, routes, fig)
else:
    print('No solution found.') 

fig

ZeroDivisionError: division by zero

In [ ]:
## GOOGLE DISTANCE MATRIX API
cumul_dist,cumul_time=0,0
for route in routes:
    total_distance,total_time = 0,0
    prev_node = None
    curr_node = None
    for node in routes[route]:
        if prev_node==None:
            prev_node = node
            continue
        curr_node = node
        prev_coord = (df_pending.iloc[prev_node]['Latitude'],df_pending.iloc[prev_node]['Longitude'])
        curr_coord = (df_pending.iloc[curr_node]['Latitude'],df_pending.iloc[curr_node]['Longitude'])
        total_distance += get_distance(prev_coord,curr_coord)
        total_time += get_travel_time(prev_coord,curr_coord)
#         print(prev_coord, end='--')
        prev_node = curr_node
#     print(curr_coord)
    print(total_distance/1000,total_time)
    cumul_dist+=total_distance
    cumul_time+=total_time
print(cumul_dist/1000,cumul_time)

In [ ]:
route_1 = routes[0]
optimized_path = []
for i in route_1:
#     print(df_pending.iloc[i,:].Latitude)
    optimized_path.append((df_pending.iloc[i,:].Latitude,df_pending.iloc[i,:].Longitude))

In [ ]:
for i in optimized_path:
    print(str(i[0])+','+str(i[1]),end=',\n')

In [ ]:
# planned_ids=[]
# for route in routes:
#     for node in routes[route]:
#         if df_pending.index[node]//100 != 0:
#             planned_ids.append(df_pending.index[node])